# Descriptors: secret weapon of frameworks

A talk presented at [Python Cerrado / Plone Conference 2024](https://2024.ploneconf.org/en/schedule/talks/descriptors-secret-weapon-of-frameworks)

## Motivating example

<img src="bulk_food_by_Dan_Bruell.jpg" alt="Bulk food bins, by Dan Bruell" width="600"> 

### Class for an item in an order of bulk food

Imagine an app for a store that sells organic food in bulk,
where customers can order nuts, dried fruit, or cereals by weight.
In that system, each order would hold a sequence of line items,
and each line item could be represented by an instance of a
class like this:

In [1]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

That's nice and simple:

In [2]:
raisins = LineItem('Golden raisins', 10, 6.95)
raisins.subtotal()

69.5

But there's a problem:

In [3]:
raisins.weight = -20
raisins.subtotal()

-139.0

This is a toy example, but not as fanciful as you may think.
Here is a story from the early days of Amazon.com:

>  We found that customers could order a negative quantity of books! And we would
  credit their credit card with the price and, I assume, wait around for them to ship the
  books.<br>
  — Jeff Bezos, founder and CEO of Amazon.com, interviewed by Wall Street Journal in “Birth of a Salesman” (October 15, 2011)

### Type hints won't save us

In [12]:
from dataclasses import dataclass

@dataclass
class LineItem:
    description: str
    weight: float
    price: float

    def subtotal(self):
        return self.weight * self.price

There is no way to specify that `weight` must be a `float` greater than zero.

Only [dependend types](https://en.wikipedia.org/wiki/Dependent_type) could fix this,
but that's a feature that exists only in obscure, academic functional languages like Agda and Idris.

### Properties work but are verbose

Properties enable the implementation of getters and setters without
changing the public interface of a class that previously
allowed reading and writing public attributes via `objec.attribute` notation.

In [9]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight  # (1)
        self.price = price

    def subtotal(self):
        return self.weight * self.price

    @property  # (2)
    def weight(self):  # (3)
        return self.__weight  # (4)

    @weight.setter  # (5)
    def weight(self, value):
        if value > 0:
            self.__weight = value  # (6)
        else:
            raise ValueError('weight must be > 0')  # (7)
            
    @property
    def price(self):
        return self.__price

    @price.setter  # (5)
    def price(self, value):
        if value > 0:
            self.__price = value
        else:
            raise ValueError('price must be > 0')

Uncomment to see a demonstration:

In [11]:
# raisins = LineItem('Golden raisins', -10, 6.95)